In [1]:
%pip install base58
%pip install cryptography

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePublicKey, EllipticCurve, SECP256K1

import base58

# 비트코인 지갑에서 비밀키 추출하기 (사전 분석)

문제에서 주어져 있는 (깨진) JSON 파일에서 공개키를 다음과 같이 얻는다.

In [3]:
xpubkey = base58.b58decode("xpub661MyMwAqRbcFwkbijMsskkrPEja9rZQAvGavNLGpthpwzbPyBDjCFUiLHVQXED2YM9pUAC7zz62ShWRPRdwbyyWEQ5CK1yP5vPWrmGCg7D")

[BIP-0032의 직렬화 형식](https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki#Serialization_format)에 따라 직렬화된 공개키를 살펴보자.

In [4]:
print(f"version bytes (4B)\t {' '.join([f'{x:02x}' for x in xpubkey[0:4]])}")
print(f"depth (1B)\t\t {' '.join([f'{x:02x}' for x in xpubkey[4:5]])}")
print(f"fingerprint (4B)\t {' '.join([f'{x:02x}' for x in xpubkey[5:9]])}")
print(f"child number (4B)\t {' '.join([f'{x:02x}' for x in xpubkey[9:13]])}")
chain_code = xpubkey[13:45]
print(f"chain code (32B)\t {' '.join([f'{x:02x}' for x in chain_code])}")
public_key_data = xpubkey[45:78]
print(f"public key data (33B)\t {' '.join([f'{x:02x}' for x in public_key_data])}")

version bytes (4B)	 04 88 b2 1e
depth (1B)		 00
fingerprint (4B)	 00 00 00 00
child number (4B)	 00 00 00 00
chain code (32B)	 8c d2 ba bc c1 f6 71 bc 33 4e aa 09 13 a4 b3 10 d1 3e 40 8f a0 11 b8 63 6a 30 f1 2d 14 04 c6 b4
public key data (33B)	 03 fd 64 98 36 81 3f a9 b2 22 2e 6f 6a 4c 92 fa f6 42 e8 36 6e c6 5d 63 40 15 17 99 d8 43 69 95 33


Depth, fingerprint, child number 모두 0으로 채워져 있고, 주어진 정보는 master key의 것임을 알 수 있다.

Public key data를 SEC-1에 따라 파싱해보자.

In [5]:
curvepoint = EllipticCurvePublicKey.from_encoded_point(SECP256K1(), public_key_data)
curvepoint.public_numbers()

<EllipticCurvePublicNumbers(curve=secp256k1, x=114612885932937541895952981204780521972488172672556735824743693348595397399859, y=36275940037485766249167729003024982632875105444299785428776410948186178771039>

이 문제에서는 이 공개키에 대응하는 비밀키를 구하면 된다. BIP-0032의 ["Master key generation"](https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki#master-key-generation)에 따르면 마스터키는 다음과 같은 과정으로 생성된다.

1. 어떤 길이의 Seed `S`를 생성한다. (이 문제에서는 여기서 BIP-0039를 따랐다.)
2. 64byte sequece `I = HMAC_SHA512(key="Bitcoin seed", data=S)`를 계산한다.
3. `I`의 첫 32byte를 `I_L`, 다음 32byte를 `I_R`이라고 한다.
4. `I_L`은 비밀키, `I_R`은 chain code로 사용된다.

방금 chain code를 `xpubkey`에서 구했으므로, `I_R`을 구한 것과 마찬가지이다.

In [6]:
I_R = chain_code